<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-IFC-Dashboard-Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# IFC Dashboard Starter Template

A minimal template to get started building IFC data dashboards.

**What this does:**
- Load an IFC file from GitHub
- Extract element types and building storeys
- Create a simple 2-tab dashboard


## Step 1: Install packages


In [1]:
%pip install -q ifcopenshell pandas plotly dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 39.4 MB/s eta 0:00:00


## Step 2: Load IFC file from GitHub


In [7]:
import urllib.request
import ifcopenshell

# ✅ Correct raw GitHub URL
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-D/main/TEST%20LZ/2024.037%20Abzugk%C3%B6rper_Aushub%20Gesamt.ifc'
local_filename = '2024.037 Abzugkörper_Aushub Gesamt.ifc'

print("📥 Downloading IFC file...")
urllib.request.urlretrieve(github_url, local_filename)
print("✅ Downloaded successfully!")

# Open IFC file
model = ifcopenshell.open(local_filename)
print(f"\n🏗️ Loaded: {model.schema}")
print(f"📊 Total elements: {len(model.by_type('IfcElement'))}")



📥 Downloading IFC file...
✅ Downloaded successfully!

🏗️ Loaded: IFC2X3
📊 Total elements: 21


## Step 3: Parse IFC data

Extract elements and their building storeys.


In [10]:
import ifcopenshell
import ifcopenshell.util.element
import pandas as pd

# --- Helper: Material auslesen ---
def get_material(element):
    try:
        mats = ifcopenshell.util.element.get_material(element)
        if isinstance(mats, list):
            return ", ".join([m.Name for m in mats if hasattr(m, "Name")])
        elif mats and hasattr(mats, "Name"):
            return mats.Name
    except Exception:
        return "Unknown"
    return "Unknown"

# --- Helper: Volumen (m³) auslesen ---
def get_volume(element):
    try:
        psets = ifcopenshell.util.element.get_psets(element)
        for pset, props in psets.items():
            for pname, val in props.items():
                if "Volume" in pname and isinstance(val, (int, float)):
                    return round(val, 3)
    except Exception:
        pass
    return None

# --- Elemente parsen ---
element_data = []

for element in model.by_type('IfcElement'):
    element_data.append({
        "ElementId": element.GlobalId,
        "ElementType": element.is_a(),
        "Material": get_material(element),
        "Volume_m3": get_volume(element)
    })

# --- DataFrame erstellen ---
df = pd.DataFrame(element_data)

print(f"✅ Parsed {len(df)} elements")
print(f"🧱 {df['Material'].nunique()} materials detected")

df.head()




✅ Parsed 21 elements
🧱 3 materials detected


,ElementId,ElementType,Material,Volume_m3
0,1q_a4kqWP7_RPAZ$2vPlfi,IfcBuildingElementProxy,Deckschicht,193.483
1,1q_a4kqWP7_RPAZ$2vPlxg,IfcBuildingElementProxy,Aufüllung,165.635
2,1q_a4kqWP7_RPAZ$2vPmDe,IfcBuildingElementProxy,Moräne,4.536
3,1q_a4kqWP7_RPAZ$2vPn5b,IfcBuildingElementProxy,Deckschicht,326.160
4,1q_a4kqWP7_RPAZ$2vPnNZ,IfcBuildingElementProxy,Aufüllung,461.848


## Step 4: Create aggregations for visualization


In [11]:
# 🔹 Gesamtvolumen pro Material berechnen
material_volume = (
    df.groupby('Material', dropna=False)['Volume_m3']
    .sum()
    .reset_index(name='TotalVolume_m3')
    .sort_values('TotalVolume_m3', ascending=False)
)

# 🔹 Elementanzahl pro Material
material_count = (
    df.groupby('Material', dropna=False)['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
)

# 🔹 Beides kombinieren
material_summary = pd.merge(material_volume, material_count, on='Material')

print("📊 Aggregation completed!")
material_summary.head(10)



📊 Aggregation completed!


,Material,TotalVolume_m3,ElementCount
0,Aufüllung,4534.385,7
1,Moräne,4178.385,7
2,Deckschicht,3533.154,7


## Step 5: Build a simple dashboard


In [13]:
from dash import Dash, dcc, html
import plotly.express as px

# --- Aggregation: Gesamtvolumen pro Material ---
material_summary = (
    df.groupby('Material', dropna=False)
    .agg(TotalVolume_m3=('Volume_m3', 'sum'))
    .reset_index()
    .sort_values('TotalVolume_m3', ascending=False)
)

# --- Plotly Diagramm ---
fig = px.bar(
    material_summary,
    x='Material',
    y='TotalVolume_m3',
    title='Gesamtvolumen pro Material',
    text_auto='.2f'
)

fig.update_layout(
    xaxis_title='Material',
    yaxis_title='Gesamtvolumen (m³)',
    xaxis_tickangle=-45,
    plot_bgcolor='white',
    title_x=0.5
)

# --- Dash App ---
app = Dash(__name__)

app.layout = html.Div([
    html.H1('🏗️ IFC Materialvolumen Dashboard'),
    html.P('Gesamtvolumen der Materialien aus dem IFC-Modell'),
    dcc.Graph(figure=fig)
])

print("🚀 Starte Dashboard...")
app.run(jupyter_mode='inline', height=700, port=8050)




🚀 Starte Dashboard...


<IPython.core.display.Javascript object>

## Next Steps

**Extend this template:**
- Add more tabs for different analyses
- Include materials data
- Add filters with dropdowns
- Try different chart types

**Try other files:**
- Change the `github_url` to load different IFC files
- Upload your own IFC files using file upload widgets

**Learn more:**
- [IFCOpenShell Documentation](https://docs.ifcopenshell.org/)
- [Plotly Dash Documentation](https://dash.plotly.com/)
- [Full example: BFH-25-Tabbed-Dashboard.ipynb](https://github.com/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-Tabbed-Dashboard.ipynb)
